<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250> 
<b>Introduction to the LSST data Butler</b> <br>
Last verified to run on <b>TBD</b> with LSST Science Pipelines release <b>TBD</b> <br>
Contact author: Alex Drlica-Wagner <br>
Credit: Originally developed in the context of the LSST Stack Club <br>
Target audience: All DP0 delegates. <br>
Container Size: medium <br>
<br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

This notebook provides an introduction to the use of the data Butler. The Butler is the LSST Science Pipelines interface for managing, reading, and writing datasets. The Butler can be used to explore the contents of the DP0.1 data repository and access the DP0.1 data. The current version of the Butler (referred to as "Gen-3") is still under development, and this notebook may be modified in the future.

The goals of this notebook are:

1. Create a Butler.
2. Programmatically explore the content of the DP0.1 data repository
3. Grab some DP0.1 data! 
4. Grab a cutout of the DP0.1 coadds at a specific (RA,Dec).

**Credit:** This notebook was originally developed by Alex Drlica-Wagner in the context of the LSST Stack Club.

### Setup

In [ ]:
# This should match the verified version listed at the start of the notebook
! eups list -s lsst_distrib

### 1. Create the Butler

In this section we are going to create an instance of the Butler. We start with some general python package imports.

In [ ]:
# Generic imports
import os,glob
import pylab as plt
plt.rcParams['figure.figsize'] = (8.0, 8.0)

We import several packages from the LSST Science Pipelines. The first import gives us access to the Butler, while the second provides tools for displaying data. More details on data display can be found in [03_Intro_to_AFW_Display.ipynb](03_Intro_to_AFW_Display.ipynb).

In [ ]:
# Stack imports
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

To create the Butler, we need to provide it with a path to the data set, which is called a "data repository". Butler repositories can be remote (i.e., pointing to an S3 bucket) or local (i.e., pointing to a directory on the local file system). In this case, we point to an S3 bucket.

In [ ]:
repo='s3://butler-us-central1-dp01'
butler = dafButler.Butler(repo)

### 2. Explore a data repository

Now that we've created an instance of the Butler, we can access the data `registry` (a database containing information about available data products). The registry will help us examine what collections of data products exist.

In [ ]:
registry = butler.registry

# We can examine the registry with
#help(registry)

The `registry` is a good tool for investigating a repository (more on the registry schema can be found [here](https://dmtn-073.lsst.io/)). For example, we can get a list of all collections, with

In [ ]:
for c in sorted(registry.queryCollections()):
    print(c)

This is our first glimpse at the data contained in the repository, but it doesn't teach us *which* collection we are actually interested in. The names do give us some hints though...

* `calib` - refers to calibration products that are used for instrument signature removal
* `refcats` - refers to the reference catalogs used for astrometric and photometric calibration
* `skymaps` - are the geometric representations of the sky coverage
* `2.2i` - refers to the processing run of the LSST DESC DC2 data (the `i` stands for `imSim`)

Collections can be nested, so we can access to everything for DC2 Run 2.2i (the primary DP0.1 data set) by selecting the collection `2.2i/runs/DP0.1`. This is a pointer to other collections that expand out recursively... More on collections can be found here: https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/organizing.html#collections

In [ ]:
# If this collection is a pointer to other collections, expand those out recursively.
collection="2.2i/runs/DP0.1"
print(collection)
for c in sorted(registry.queryCollections(collection,flattenChains=True)):
    print(c, registry.getCollectionType(c))

We now create a new Butler instance specifying that we are specifically interested in the `2.2i/runs/DP0.1` data collection. For most uses, this will be the line you will use to create a Butler to work on DP0.1, since you now know that this collection exists.

In [ ]:
# Create a new butler with the collection of interest
butler = dafButler.Butler(repo,collections=collection)
registry = butler.registry

You are probably interested in figuring out what kind of data is present in DP0.1. The LSST Science Pipelines classify different data products in terms of `DatasetTypes`. However, individual `DatasetTypes` are defined globally, and thus don't belong to a specific collection. Thus, a query of DatasetTypes will return all DatasetTypes belonging to the repository, and not all of them may belong to the collection of interest.

In [ ]:
for x in sorted(registry.queryDatasetTypes()):
    print(x)

Again, this list may seem a bit overwhelming, but we can extract some information from the names of the dataset types:

- `calexp` - Refers to individual calibrated exposure
- `deepCoadd` - Refers to products produced on the coadd images (both images and and source catalogs)
- `src` - refers the the catalog of sources
- `skyMap` - refers to geometric representations of the sky coverage

We access specific data sets through a set of specifications known as a data identifier (`dataId`). Each `DatasetType` can be identified with a different set of properties, so it is important to be able to determine what properties need to be specified to access data of a specific type. It is possible to get all `DatasetRef` (which include the `dataId`) for a specific `datasetType` in a specific collection with a query like this. Note that this doesn't necessarily guarentee that the specific data set exists, so we include a check that the data set has a valid Uniform Resource Identifier (URI).

In [ ]:
datasetRefs = registry.queryDatasets(datasetType='calexp',collections=collection)
for i,ref in enumerate(datasetRefs):
    print(ref.dataId.full)
    try: uri = butler.getURI(ref)
    except: print("File not found...")
    if i > 10: break

We can get the path from the URI that is returned by `butler.getURI`. Note that this URI does not have to refer to a local path on the filesystem.

In [ ]:
print("Repo path: ",repo)
uri = butler.getURI(ref)
print("File URI: ",uri)
!ls -lh {uri.ospath}

Now say we want to restrict our selection to datasets associated with a specific filter. We can add that constraint to our query, but first we need to figure out what the filters are called... Looking at the dataId object, we see the attributes `physical_filter` and `band` look promising.

In [ ]:
ref.dataId.full

In [ ]:
print(f"physical_filter = {ref.dataId['physical_filter']}")
print(f"band = {ref.dataId['band']}")

It looks like `band` is what we want, so we put it in the `where` argument of `queryDatasets`. Let's try the $g$ band.

In [ ]:
# You can also sub-select on specific properties of a data set
datasetRefs = registry.queryDatasets(datasetType='calexp',dataId={'band': 'g'}, where='visit > 700000', collections=collection)
for i,ref in enumerate(datasetRefs):
    print(ref.dataId.full)
    try: uri = butler.getURI(ref)
    except: print("File not found...")
    if i > 10: break

### 3. Get some data

Ok, now we have all the information we need to ask the Butler to get a specific data product. We have identified a collection (`2.2i/runs/DP0.1`), a `datasetType` (`calexp`), and the `dataId` (from the `datasetRef`) to uniquely specify an instance of this data set.

From the list above, let's choose one detector from one visit and ask the Butler to get a `calexp` for us.

In [ ]:
# We could pass the datasetRef that we found above, but since the query may 
# return results in a different order we define the dataId explicitly for reproducibility. 
dataId = {'visit': '703697', 'detector': 80, 'band':'g'}
calexp = butler.get('calexp',dataId=dataId)

The `calexp` is a calibrated CCD from a single exposure. We'll use the afwDisplay interface to show the pixel values and mask plane (more on afwDisplay can be found in other notebooks).

In [ ]:
fig = plt.figure()
display = afwDisplay.Display()
display.scale('linear', 'zscale')
display.mtv(calexp)
plt.show()

Now that we have a calibrated image, we may want to get the catlog of sources that were extracted from it. The `src` catalog associated with this `calexp` we pass the `dataId` to the butler with the `src` datasetType. Note that this performs another query to the registry database to find the `src` catalog that matches our dataId requirements.

In [ ]:
# We could get the src table using the dataId as we did above for the calexp, 
# but this would require the butler to perform another query of the database. 
# Instead, we can just pass the ref itself directly to butler.get
src = butler.get('src',dataId)
src = src.copy(True)
src.asAstropy()

We can now plot the `calexp` with the `src` catalog overlaid. We leave an investigation of this image as an exercise to the user :)

In [ ]:
# And plot!
fig = plt.figure()
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(calexp)
plt.gca().axis('off')

with afw_display.Buffering():
    for s in src:
        afw_display.dot('+', s.getX(), s.getY(), ctype=afwDisplay.RED)
        afw_display.dot('o', s.getX(), s.getY(), size=20, ctype='orange') 

### 4. Querying for multiple data sets

In the case above, both the `calexp` and `src` can be found by the registry, but this will not always necessarily be the case. The `queryDimensions` method provides a more flexible way to query for multiple datasets (requiring an instance of all datasets to be available for that dataId) or ask for different dataId keys than what is used to identify the dataset (which invokes various built-in relationships). An example of this is provided below:

In [ ]:
# Use queryDimensions to provide more flexible access
dataIds = registry.queryDataIds(["visit", "detector", "band"], datasets=["calexp","src"], where='visit = 703697',
                                collections=collection)
for i,dataId in enumerate(dataIds):
    print(dataId.full)
    if i > 10: break

In [ ]:
# Use queryDataIds to grab the dataIds for a subset
dataIds = registry.queryDataIds(["visit", "detector"], datasets=["calexp","src"], 
                                where="band='g' and detector=0 and visit > 700000",
                                collections=collection)
for i,dataId in enumerate(dataIds):
    print(dataId.full)
    if i > 10: break

You can also get more metadata about a data product dimension. (Note that the record for an exposure and a visit are different.)

In [ ]:
for dim in ['exposure','visit','detector']:
    print(list(registry.queryDimensionRecords(dim, where='visit = 971990 and detector=0'))[0])
    print()

### 5. Grabbing an Image Cutout

Say we want to grab a cutout of the DP0.1 coadded images at a specific location. In order to do this, we need a few other packages from the LSST Science Pipelines. In particular, access to the geometry and coordinate packages.

In [2]:
import lsst.geom as geom
import lsst.afw.coord as afwCoord

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given afw SpherePoint radec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.
    
    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, degrees
    dec: float
        Declination of the center of the cutout, degrees
    filter: string 
        Filter of the image to load
    datasetType: string ['deepCoadd']  
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional] 
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional] 
        Side of the cutout region in pixels.
    
    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, afwGeom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")
    
    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize//2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox':bbox}
    
    cutout_image = butler.get(datasetType, parameters=parameters, immediate=True, dataId=coaddId)
    
    return cutout_image

In [ ]:
ra,dec = 55.064, -29.783 # center of DC2 region
cutout_image = cutout_coadd(butler,ra,dec,datasetType='deepCoadd', cutoutSideLength=201)
print("The size of the cutout is: ",cutout_image.image.array.shape)

In [ ]:
fig = plt.figure()
afw_display = afwDisplay.Display(1)
afw_display.scale('asinh', 'zscale')
afw_display.mtv(cutout_image.image)
plt.gca().axis('off')